In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
client = Client(n_workers=16)
client

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:41685 Dashboard: http://127.0.0.1:42711/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


In [3]:
!pwd

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker


In [4]:
data_loc = 'instacart_2017_05_01/'

In [5]:
aisles = dd.read_csv(data_loc + 'aisles.csv', header=0, enforce=True)
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [6]:
depts = dd.read_csv(data_loc + 'departments.csv', header=0, enforce=True)
depts.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [7]:
order_products = dd.read_csv(data_loc + 'order_products*.csv', header=0)
order_products.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, order_id to reordered
dtypes: int64(4)

In [8]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [9]:
products = dd.read_csv(data_loc + 'products.csv')
products.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, product_id to department_id
dtypes: object(1), int64(3)

In [10]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [11]:
order_products = dd.merge(products, order_products)
order_products.head()

,product_id,product_name,aisle_id,department_id,order_id,add_to_cart_order,reordered
0,1,Chocolate Sandwich Cookies,61,19,1107,7,0
1,1,Chocolate Sandwich Cookies,61,19,5319,3,1
2,1,Chocolate Sandwich Cookies,61,19,7540,4,1
3,1,Chocolate Sandwich Cookies,61,19,9228,2,0
4,1,Chocolate Sandwich Cookies,61,19,9273,30,0


In [12]:
order_products = order_products.persist()

In [13]:
order_products_counts = order_products['product_name'].value_counts().compute()

In [14]:
order_products_counts[:100]

Banana                                                            491291
Bag of Organic Bananas                                            394930
Organic Strawberries                                              275577
Organic Baby Spinach                                              251705
Organic Hass Avocado                                              220877
Organic Avocado                                                   184224
Large Lemon                                                       160792
Strawberries                                                      149445
Limes                                                             146660
Organic Whole Milk                                                142813
Organic Raspberries                                               142603
Organic Yellow Onion                                              117716
Organic Garlic                                                    113936
Organic Zucchini                                   

In [22]:
depts.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [21]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [118]:
%%time
# Unfortunately, Dask does not support apply for columns,
# so we convert Dask Dataframe to Pandas Dataframe first.
grouped_df = order_products.groupby(['aisle_id'])[['reordered']] \
            .count() \
            .compute()

CPU times: user 226 ms, sys: 33.7 ms, total: 260 ms
Wall time: 1.57 s


In [119]:
grouped_df = grouped_df \
            .reset_index() \
            .rename(columns={'reordered': 'counts'}) \
            .sort_values(by='counts', ascending=False)
grouped_df.head()

,aisle_id,counts
23,24,3792661
82,83,3568630
122,123,1843806
119,120,1507583
20,21,1021462


In [120]:
grouped_df['%'] = grouped_df['counts'] / sum(grouped_df['counts']) * 100
grouped_df['cumulative %'] = grouped_df['%'].cumsum()

pd.set_option('display.max_rows', None)  # Unlimited rows
grouped_df.head(grouped_df.shape[0])

,aisle_id,counts,%,cumulative %
23,24,3792661,11.214551,11.214551
82,83,3568630,10.552112,21.766663
122,123,1843806,5.451966,27.218629
119,120,1507583,4.457785,31.676414
20,21,1021462,3.020370,34.696784
83,84,923659,2.731175,37.427959
114,115,878150,2.596609,40.024568
106,107,753739,2.228737,42.253305
90,91,664493,1.964845,44.218150
111,112,608469,1.799187,46.017337


In [121]:
grouped_df = grouped_df.merge(aisles.compute(), 
                              on='aisle_id')
grouped_df = grouped_df.nlargest(n=3000, columns='counts')

In [122]:
grouped_df.head(10)

,aisle_id,counts,%,cumulative %,aisle
0,24,3792661,11.214551,11.214551,fresh fruits
1,83,3568630,10.552112,21.766663,fresh vegetables
2,123,1843806,5.451966,27.218629,packaged vegetables fruits
3,120,1507583,4.457785,31.676414,yogurt
4,21,1021462,3.020370,34.696784,packaged cheese
5,84,923659,2.731175,37.427959,milk
6,115,878150,2.596609,40.024568,water seltzer sparkling water
7,107,753739,2.228737,42.253305,chips pretzels
8,91,664493,1.964845,44.218150,soy lactosefree
9,112,608469,1.799187,46.017337,bread


In [130]:
aisle_df = grouped_df \
            .set_index("aisle") \
            .drop("cumulative %", axis = 1)
aisle_df

,aisle_id,counts,%
aisle,,,
fresh fruits,24,3792661,11.214551
fresh vegetables,83,3568630,10.552112
packaged vegetables fruits,123,1843806,5.451966
yogurt,120,1507583,4.457785
packaged cheese,21,1021462,3.020370
milk,84,923659,2.731175
water seltzer sparkling water,115,878150,2.596609
chips pretzels,107,753739,2.228737
soy lactosefree,91,664493,1.964845
